In [4]:
import psutil
import gc
import os
# import pandas as pd
import zipfile
import numpy as np
import nibabel as nib
from io import BytesIO
# import matplotlib.pyplot as plt
import nilearn as nil # https://nilearn.github.io/stable/quickstart.html
import time
from nilearn import plotting
from tqdm import tqdm

from multiprocessing import Pool
import torch
from torch.utils import data

import io
import ants

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [34]:
from PIL import Image
import matplotlib.pyplot as plt

In [48]:
data_path = '/scratch/users/neuroimage/conda/data/preprocessed/imgss'
meta_data_path = '/scratch/users/neuroimage/conda/data/ADNI1_Complete_2Yr_3T_4_18_2023.csv'

In [10]:
df = pd.read_csv(meta_data_path)
df.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I205567,136_S_1227,MCI,F,66,m18,MRI,MPR; ; N3; Scaled,Processed,9/29/2008,NiFTI,4/12/2023
1,I66824,136_S_1227,MCI,F,65,bl,MRI,MPR; ; N3; Scaled,Processed,2/21/2007,NiFTI,4/12/2023
2,I79080,136_S_1227,MCI,F,65,m06,MRI,MPR; ; N3; Scaled,Processed,9/11/2007,NiFTI,4/12/2023
3,I143856,136_S_1227,MCI,F,67,m24,MRI,MPR; ; N3; Scaled,Processed,3/23/2009,NiFTI,4/12/2023
4,I99265,136_S_1227,MCI,F,66,m12,MRI,MPR; ; N3; Scaled,Processed,3/06/2008,NiFTI,4/12/2023


In [12]:
df_new = df[['Image Data ID', 'Subject', 'Group']].set_index(df[['Image Data ID', 'Subject', 'Group']].apply(lambda x: (x['Image Data ID'], x['Subject']), axis=1))
df_new.head()

,Image Data ID,Subject,Group
"(I205567, 136_S_1227)",I205567,136_S_1227,MCI
"(I66824, 136_S_1227)",I66824,136_S_1227,MCI
"(I79080, 136_S_1227)",I79080,136_S_1227,MCI
"(I143856, 136_S_1227)",I143856,136_S_1227,MCI
"(I99265, 136_S_1227)",I99265,136_S_1227,MCI


In [17]:
df_dict = df_new['Group'].to_dict()
# df_dict

In [51]:
# pair the image with labels and put them into respective folder
out_dir = '/scratch/users/neuroimage/conda/data/prepared_data/'
for filename in os.listdir(data_path):
    file_path = os.path.join(data_path, filename)
    if os.path.isfile(file_path) and filename.endswith('gm.nii.gz'):
        split_str = filename.split('-')
        # print(split_str)
        img_id = split_str[2]
        subject = split_str[1]
        assert (img_id, subject) in df_dict
        class_name = df_dict[(img_id, subject)]
        # print(type(category))
        print(os.path.join(data_path, filename))
        img = nib.load(os.path.join(data_path, filename))
        print(img)
        
        data = img.get_fdata()
        print(data.shape)
        print(filename)
        plt.imshow(data[-1,:,:], cmap='gray')
        z_shape = data.shape[2]
        for i in range(z_shape):
            np_array = data[:,:,i]
            img = Image.fromarray(np.uint8(np_array))
            filename_new = filename.split('.')[0] + f'_{str(i)}.jpg'
            # print(filename_new)
            # print(os.path.join(out_dir, class_name, filename_new)) 
            img.save(os.path.join(out_dir, class_name, filename_new))
        print('finish', filename)


/scratch/users/neuroimage/conda/data/preprocessed/imgss/pp-023_S_1126-I94863-gm.nii.gz

<class 'nibabel.nifti1.Nifti1Image'>
data shape (121, 145, 121)
affine:
[[  0.    0.   -1.5  -0. ]
 [  0.   -1.5   0.   -0. ]
 [ -1.5   0.    0.  180. ]
 [  0.    0.    0.    1. ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 121 145 121   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.  1.5 1.5 1.5 0.  0.  0.  0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin          

In [6]:
# sample
filename = 'pp-051_S_1331-I82509-gm.nii.gz'
img = nib.load(os.path.join(data_path, filename))

data = img.get_fdata()

In [60]:
from torchvision import datasets, transforms
batch_size = 32

data_transform = {
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

full_dataset = datasets.ImageFolder(out_dir, transform=data_transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)

In [61]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])


In [56]:
train_dataset

In [57]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models

In [58]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# dataloader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# resnet
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/scratch/users/neuroimage/conda/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/scratch/users/neuroimage/conda/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.



AttributeError: 'Subset' object has no attribute 'classes'

In [ ]:
# train
num_epochs = 10
for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0
    train_corrects = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)

    model.eval()
    test_loss = 0.0
    test_corrects = 0
    with torch.set_grad_enabled(False):
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects.double() / len(train_dataset)
    test_loss = test_loss / len(test_dataset)
    test_acc = test_corrects.double() / len(test_dataset)
    print('Epoch [{}/{}], train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}'.format(
        epoch+1, num_epochs, train_loss, train_acc, test_loss, test_acc))
